In [1]:
# uncomment preferred option
#!pip install -q git+https://github.com/deepmipt/bert.git@feat/keras deeppavlov tensorflow-gpu==1.14.0
#!pip install -q git+https://github.com/deepmipt/bert.git@feat/keras deeppavlov tensorflow-gpu==2.0.0b1

In [2]:
import tensorflow as tf
import numpy as np

v = str(tf.__version__).split('.')[:2]
assert int(v[0]) > 1 or (int(v[1]) >= 13), 'this notebook was tested with TF v1.14.0 and v2.0.0-beta1'
tf.compat.v1.enable_eager_execution()
is_eager = tf.compat.v1.executing_eagerly()
is_eager

True

In [3]:
from bert_dp import bert
from bert_dp.metrics import F1Score
BATCH_SIZE = 64

In [4]:
tf.keras.utils.get_file(
    fname='paraphraser_rubert_v0.tar.gz',
    origin='http://files.deeppavlov.ai/deeppavlov_data/bert/paraphraser_rubert_v0.tar.gz',
    cache_subdir='models',
    extract=True,
    cache_dir='.'
)

'./models/paraphraser_rubert_v0.tar.gz'

## Reference model and data from DeepPavlov

In [5]:
from deeppavlov.dataset_readers.paraphraser_reader import ParaphraserReader
ds = ParaphraserReader().read(data_path='dataset', do_lower_case=False)
texts_a = []
texts_b = []
labels = []
for (ta, tb), label in ds['test']:
    texts_a.append(ta)
    texts_b.append(tb)
    labels.append(label)

In [6]:
from deeppavlov import build_model, configs

para_model = build_model(configs.classifiers.paraphraser_rubert, download=True)

bert_preprocessor = para_model[0]
bert_classifier = para_model[1]

2019-06-21 14:22:28.839 INFO in 'deeppavlov.download'['download'] at line 116: Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v1.tar.gz download because of matching hashes
I0621 14:22:28.839187 139961005266752 download.py:116] Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v1.tar.gz download because of matching hashes
2019-06-21 14:22:29.752 INFO in 'deeppavlov.download'['download'] at line 116: Skipped http://files.deeppavlov.ai/deeppavlov_data/classifiers/paraphraser_rubert_v0.tar.gz download because of matching hashes
I0621 14:22:29.752000 139961005266752 download.py:116] Skipped http://files.deeppavlov.ai/deeppavlov_data/classifiers/paraphraser_rubert_v0.tar.gz download because of matching hashes
2019-06-21 14:22:29.767 INFO in 'deeppavlov.download'['download'] at line 116: Skipped http://files.deeppavlov.ai/datasets/paraphraser.zip download because of matching hashes
I0621 14:22:29.767950 139961005266752 d

In [7]:
features_array = bert_preprocessor(texts_a, texts_b)
x_test = np.array([f.input_ids for f in features_array])
y_test = np.array(labels)

In [8]:
p = bert_classifier(features_array)

## tf.keras.Model construction and compilation for evaluation

In [9]:
class BERTClassifierCompat(tf.keras.Model):  # bert.BERT):
    """Subclassed model for classification compatible with official checkpoints from Google."""
    def __init__(self, num_classes=2, **kwargs):
        kwargs['name'] = 'bert'
#         with tf.name_scope('bert'):
        super().__init__(**kwargs)
        self.num_classes = num_classes
        self.bert = bert.BERT(name='bert')
        self.dropout = tf.keras.layers.Dropout(rate=0.5)
        self.softmax = tf.keras.layers.Activation(activation='softmax')
        
    def build(self, batch_input_shape):
        self.output_weights = self.add_weight(shape=(self.num_classes, 768),
                                              dtype=tf.float32,
                                              initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02),
                                              name="output_weights")
        self.output_bias = self.add_weight(shape=(self.num_classes),
                                           dtype=tf.float32,
                                           initializer=tf.keras.initializers.Zeros(),
                                           name='output_bias')
        super().build(batch_input_shape)
        
    def call(self, token_ids, training=None, mask=None, **kwargs):
        po = self.bert(token_ids, training=training, mask=mask)
        po = self.dropout(po, training=training)
        int_logits = tf.matmul(po, self.output_weights, transpose_b=True)
        logits = tf.nn.bias_add(int_logits, self.output_bias)
        out = self.softmax(logits)
        return out

In [10]:
paraphrase_detector = BERTClassifierCompat()
# we build the model here rather than simply calling it in order to properly recreate variable names
paraphrase_detector.build(batch_input_shape=(BATCH_SIZE, None))
r1 = paraphrase_detector.predict(x_test[:BATCH_SIZE])

In [11]:
LEARNING_RATE = 2e-05
paraphrase_detector.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE,
                                                               epsilon=1e-6),
                            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                            metrics=[tf.keras.metrics.SparseCategoricalAccuracy(),
                                     F1Score(num_classes=2)
                                    ])

W0621 14:24:00.229381 139961005266752 deprecation.py:506] From /home/nab/PycharmProjects/work/venv/lib/python3.6/site-packages/tensorflow/python/util/deprecation.py:507: calling count_nonzero (from tensorflow.python.ops.math_ops) with axis is deprecated and will be removed in a future version.
Instructions for updating:
reduction_indices is deprecated, use axis instead


In [12]:
# baseline to ensure checkpoint loading make a difference
paraphrase_detector.evaluate(x=x_test, y=y_test, batch_size=BATCH_SIZE)

1899/1899 [==============================] - 10s 5ms/sample - loss: 0.6867 - sparse_categorical_accuracy: 0.5540 - f1_score: 0.6972


[0.68670883161134, 0.55397576, 0.69717556]

In [13]:
paraphraser_checkpoint_path = 'models/paraphraser_rubert/model_rubert'
paraphraser_saver = tf.compat.v1.train.Saver(var_list=paraphrase_detector.variables)
paraphraser_saver.restore(sess=None if is_eager else tf.keras.backend.get_session(),
                          save_path=paraphraser_checkpoint_path)

W0621 14:24:16.976696 139961005266752 saver.py:795] Saver is deprecated, please switch to tf.train.Checkpoint or tf.keras.Model.save_weights for training checkpoints. When executing eagerly variables do not necessarily have unique names, and so the variable.name-based lookups Saver performs are error-prone.


In [14]:
r2 = paraphrase_detector.predict(x_test[:BATCH_SIZE])
try:
    np.testing.assert_allclose(r1, r2)
except AssertionError:
    # check at least some variables were loaded
    print('Outputs are changed')

Outputs are changed


In [16]:
paraphrase_detector.evaluate(x=x_test, y=y_test, batch_size=BATCH_SIZE)

1899/1899 [==============================] - 9s 4ms/sample - loss: 0.3883 - sparse_categorical_accuracy: 0.8489 - f1_score: 0.8789


[0.38825273774309244, 0.84886783, 0.87885183]

From the RuBERT paper: accuracy = 84.99 +- 0.35, f1 = 87.73 +- 0.23

In [17]:
p_ = np.argmax(paraphrase_detector.predict(x_test), axis=-1)

In [18]:
all(p_ == p)

True